In [0]:
import keras.models
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import math
from keras.applications.vgg19 import VGG19
import os
from glob import glob
from scipy import stats

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My\ Drive/Data\ Sets

/content/gdrive/My Drive/Data Sets


In [0]:
init_model = VGG19(weights='imagenet', include_top=False)
model = keras.models.Sequential([
    Dense(512, activation='relu', input_dim=25088),
    Dropout(0.4),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(101, activation='softmax'),
])
model.load_weights("weights2.hdf5")
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
f = open("ucfTrainTestlist/testlist01.txt", "r")
temp = f.read()
test = pd.DataFrame()
test['name'] = temp.split('\n')
test = test[:-1]
test_videos = test['name']

In [0]:
train = pd.read_csv('train_01/train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [0]:
predict, actual = [], []
for i in tqdm(range(test_videos.shape[0])):
    videoFile = test_videos[i]
    cnt = 0
    cap = cv2.VideoCapture('UCF-101/'+videoFile.split(' ')[0])
    frameRate = cap.get(5)
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1)
        ret, frame = cap.read()
        if ret != True : break
        if frameId % frameRate == 1:
            filename ='temp/' + "_frame%d.jpg" % cnt
            cnt+=1
            cv2.imwrite(filename, frame)
    cap.release()
    images = glob("temp/*.jpg")
    prediction_images = []
    for _i in images:
        img = image.load_img(_i, target_size=(224,224,3))
        img = image.img_to_array(img)/255
        prediction_images.append(img)
    prediction_images = init_model.predict(np.array(prediction_images))
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 25088)
    prediction = model.predict_classes(prediction_images)
    predict.append(y.columns.values[stats.mode(prediction)[0][0]])
    actual.append(videoFile.split('/')[1].split('_')[1])

100%|██████████| 3783/3783 [35:31<00:00,  1.05s/it]


In [1]:
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

45.04943166798833
